## Imports

In [ ]:
import os
from glob import glob
from tqdm import tqdm

import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from scipy.integrate import simpson
from scipy.optimize import curve_fit
from scipy.signal import argrelextrema
from scipy.stats import spearmanr, median_abs_deviation

from astropy.io import fits
from astropy.convolution import convolve, Gaussian1DKernel
from astropy.stats import sigma_clipped_stats

%matplotlib inline

In [ ]:
plt.rcParams.update({'font.size': 20})

## Load Lines and Data

In [ ]:
data_path = '../data'
lines_vac, lines_air = np.loadtxt(f'{data_path}/hydroxyl_lines_rousselot_2000.txt').T

In [ ]:
results_path = '../results'
df = pd.read_csv(f'{results_path}/lines_norm_gdl_fit.csv', index_col=0)
columns = df.columns

In [ ]:
col_sub = []
for column in columns[9:28]:
    if 'sigma1_cov' in column or 'sigma2_cov' in column:
        col_sub.append(column[9:])
    else:
        col_sub.append(column.split('_')[1])

In [ ]:
col_sub = np.array(col_sub)

In [ ]:
col_sub

In [ ]:
time_centered = df['spec_mjd'] - int(df['spec_mjd'].min())

In [ ]:
mask1_1 = (time_centered > 504) & (time_centered < 504.5)
mask1_2 = (time_centered > 504.58) & (time_centered < 505.515)
mask1_3 = (time_centered > 505.58) & (time_centered < 506.578)
mask1_4 = (time_centered > 506.61) & (time_centered < 507)
mask1 = mask1_1 | mask1_2 | mask1_3 | mask1_4
mask2 = (time_centered > 543) & (time_centered < 547)

## Medians

In [ ]:
columns[:9]

In [ ]:
fs = 45
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky Medians', fontsize=fs)
plt.scatter(time_centered, df[columns[3]], label='spectrum', alpha=0.5, color='C0', s=250)
plt.vlines(505.5, df[columns[3]].min(), df[columns[3]].max(), color='C5', linewidth=5, label='Event 1')
plt.vlines(545, df[columns[3]].min(), df[columns[3]].max(), color='C7', linewidth=5, label='Event 2')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Flux', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.yscale('log')
plt.ylim(10**-9, 10**-4)
plt.legend(fontsize=fs)

In [ ]:
fs = 45
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky Medians (Event 1)', fontsize=fs)
plt.scatter(time_centered[mask1], df[columns[3]][mask1], label='Event 1', alpha=0.5, color='C5', s=250)
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Flux', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.yscale('log')
plt.ylim(10**-9, 10**-4)
plt.legend(fontsize=fs)

In [ ]:
fs = 45
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky Medians (Event 2)', fontsize=fs)
plt.scatter(time_centered[mask2], df[columns[3]][mask2], label='Event 2', alpha=0.5, color='C7', s=250)
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Flux', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.yscale('log')
plt.ylim(10**-9, 10**-4)
plt.legend(fontsize=fs)

## Bands

In [ ]:
fs = 45
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky Band Medians (Normalized)', fontsize=fs)
plt.scatter(time_centered, df[columns[4]], label='J Band', alpha=0.5, color='C1', s=250)
plt.scatter(time_centered, df[columns[5]], label='H Band', alpha=0.5, color='C2', s=250)
plt.scatter(time_centered, df[columns[6]], label='K Band', alpha=0.5, color='C3', s=250)
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Flux', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.yscale('log')
plt.ylim(10**-2, 10**1.5)
plt.legend(fontsize=fs)

In [ ]:
fs = 45
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky Band Medians (Event 1, Normalized)', fontsize=fs)
plt.scatter(time_centered[mask1], df[columns[4]][mask1], label='J Band', alpha=0.5, color='C1', s=250)
plt.scatter(time_centered[mask1], df[columns[5]][mask1], label='H Band', alpha=0.5, color='C2', s=250)
plt.scatter(time_centered[mask1], df[columns[6]][mask1], label='K Band', alpha=0.5, color='C3', s=250)
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Flux', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.yscale('log')
plt.ylim(10**-2, 10**1.5)
plt.legend(fontsize=fs)

In [ ]:
fs = 45
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky Band Medians (Event 2, Normalized)', fontsize=fs)
plt.scatter(time_centered[mask2], df[columns[4]][mask2], label='J Band', alpha=0.5, color='C1', s=250)
plt.scatter(time_centered[mask2], df[columns[5]][mask2], label='H Band', alpha=0.5, color='C2', s=250)
plt.scatter(time_centered[mask2], df[columns[6]][mask2], label='K Band', alpha=0.5, color='C3', s=250)
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Flux', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.yscale('log')
plt.ylim(10**-2, 10**1.5)
plt.legend(fontsize=fs)

## K/J

In [ ]:
fs = 45
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: K/J', fontsize=fs)
plt.scatter(time_centered, df[columns[7]], label='spectrum', alpha=0.5, color='C0', s=250)
plt.vlines(505.5, 0.271, df[columns[7]].max(), color='C5', linewidth=5, label='Event 1')
plt.vlines(545, 0.271, df[columns[7]].max(), color='C7', linewidth=5, label='Event 2')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Unitless', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.yscale('log')
plt.ylim(10**-1, 10**2)
plt.legend(fontsize=fs)

In [ ]:
fs = 45
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: K/J (Event 1)', fontsize=fs)
plt.scatter(time_centered[mask1], df[columns[7]][mask1], label='Event 1', alpha=0.5, color='C5', s=250)
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Unitless', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.yscale('log')
plt.ylim(10**-1, 10**2)
plt.legend(fontsize=fs)

In [ ]:
fs = 45
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: K/J (Event 2)', fontsize=fs)
plt.scatter(time_centered[mask2], df[columns[7]][mask2], label='Event 2', alpha=0.5, color='C7', s=250)
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Unitless', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.yscale('log')
plt.ylim(10**-1, 10**2)
plt.legend(fontsize=fs)

## Airmass

In [ ]:
fs = 45
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: Airmass', fontsize=fs)
plt.scatter(time_centered, df[columns[8]], label='spectrum', alpha=0.5, color='C0', s=250)
plt.vlines(505.5, 1, df[columns[8]].max(), color='C5', linewidth=5, label='Event 1')
plt.vlines(545, 1, df[columns[8]].max(), color='C7', linewidth=5, label='Event 2')
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Blank', fontsize=fs)
plt.tick_params(labelsize=fs)
#plt.yscale('log')
plt.ylim(0, 4)
plt.legend(fontsize=fs)

In [ ]:
fs = 45
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: Airmass (Event 1)', fontsize=fs)
plt.scatter(time_centered[mask1], df[columns[8]][mask1], label='Event 1', alpha=0.5, color='C5', s=250)
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Blank', fontsize=fs)
plt.tick_params(labelsize=fs)
#plt.yscale('log')
plt.ylim(0, 4)
plt.legend(fontsize=fs)

In [ ]:
fs = 45
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: Airmass (Event 2)', fontsize=fs)
plt.scatter(time_centered[mask2], df[columns[8]][mask2], label='Event 1', alpha=0.5, color='C7', s=250)
plt.xlabel('Days from First Observation ($MJD_0$=58327)', fontsize=fs)
plt.ylabel('Blank', fontsize=fs)
plt.tick_params(labelsize=fs)
#plt.yscale('log')
plt.ylim(0, 4)
plt.legend(fontsize=fs)